In [1]:
# general
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import os

# custom local libs
from function import baseball_lambda

Python REPL can execute arbitrary code. Use with caution.


In [2]:
# read local .env file
_ = load_dotenv(find_dotenv()) 

In [3]:
# set Langsmith project
today = datetime.now().strftime("%Y%m%d")
os.environ["LANGCHAIN_PROJECT"] = f"Baseball Curveballs - {today}"

In [4]:
session_id = '2964'

In [5]:
task =  "Consider the first week of August 2020 - find 3 pitchers who's curveballs were most similar to Max Scherzer's."
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Plan'}}
----
Distance to neareast plan: 0.5457632541656494
Formulating a new plan based on User input
Modifying plan with function metadata


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
task = """
make sure the plan follows this general flow:
1) create an average vector for each pitcher's curveball.
2) Train a knn model on this data
3) Use this model to find the 3 pitchers with the most similar curveball
"""
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

In [ ]:
task = 'approved'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

In [ ]:
task = 'yes'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

In [ ]:
task = 'yes, thank you'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)